In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# !ls

ETL.pptx        Project-2.ipynb Resources       data_etl.ipynb  queries.sql


In [4]:
import pymysql
pymysql.install_as_MySQLdb()

Context

Over 67k employee reviews for Google, Amazon, Facebook, Apple, and Microsoft
Content

This dataset contains employee reviews separated into the following categories:

    Index: index
    Company: Company name
    Location : This dataset is global, as such it may include the country's name in parenthesis [i.e "Toronto, ON(Canada)"]. However, if the location is in the USA then it will only include the city and state[i.e "Los Angeles, CA" ]
    Date Posted: in the following format MM DD, YYYY
    Job-Title: This string will also include whether the reviewer is a 'Current' or 'Former' Employee at the time of the review
    Summary: Short summary of employee review
    Pros: Pros
    Cons: Cons
    Overall Rating: 1-5
    Work/Life Balance Rating: 1-5
    Culture and Values Rating: 1-5
    Career Opportunities Rating: 1-5
    Comp & Benefits Rating: 1-5
    Senior Management Rating: 1-5
    Helpful Review Count: A count of how many people found the review to be helpful
    Link to Review : This will provide you with a direct link to the page that contains the review. However it is likely that this link will be outdated

NOTE: 'none' is placed in all cells where no data value was found.
Acknowledgements

This data was scraped from Glassdoor
Inspiration

To inspire people to create ML models to search for meaningful trends within this dataset 

### Store CSV into DataFrame

In [22]:
review_file = "Resources/employee_reviews.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head(2)

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...


In [33]:
google_df=customer_data_df[customer_data_df["company"]=="google"]
google_df.shape


(7819, 17)

### Create new data with select columns

In [4]:
new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
new_customer_data_df.head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Store JSON data into a DataFrame

In [11]:
stock_value_file = "Resources/HistoricalQuotes.csv"
stock_df = pd.read_csv(stock_value_file)
stock_df.head()

,date,close,volume,open,high,low
0,2019/04/12,1222.73,1213578.0,1215.62,1223.0500,1213.2900
1,2019/04/11,1209.59,849233.0,1208.90,1212.7400,1204.5400
2,2019/04/10,1206.45,774664.0,1205.09,1208.9377,1200.6609
3,2019/04/09,1202.69,983145.0,1201.89,1207.1500,1198.3800
4,2019/04/08,1208.28,1087408.0,1211.11,1213.9500,1204.6950


### Clean DataFrame

In [6]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database

In [11]:
rds_connection_string = "root:sifre@localhost/customer_db"
engine = create_engine(f'mysql://{rds_connection_string}')

### Check for tables

In [12]:
engine.table_names()

[]

### Use pandas to load csv converted DataFrame into database

In [13]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [14]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [15]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [16]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
